# Train a gesture recognition model for microcontroller use

## Configure dependencies

Run the following cell to ensure the correct version of TensorFlow is used.

In [ ]:
%tensorflow_version 2.x


We'll also clone the TensorFlow repository, which contains the training scripts, and copy them into our workspace.

In [ ]:
# Clone the repository from GitHub
!git clone --depth 1 -q https://github.com/tensorflow/tensorflow
# Copy the training scripts into our workspace
!cp -r /Users/ncs/project554/tensorflow2/tensorflow/lite/experimental/micro/examples/magic_wand/train train

## Prepare the data

Next, we'll download the data and extract it into the expected location within the training scripts' directory.

In [ ]:
# Download the data we will use to train the model
!wget http://download.tensorflow.org/models/tflite/magic_wand/data.tar.gz
# Extract the data into the train directory
!tar xvzf data.tar.gz -C train 1>/dev/null

We'll then run the scripts that split the data into training, validation, and test sets.

In [ ]:
# The scripts must be run from within the train directory
%cd train
# Prepare the data
!python data_prepare.py
# Split the data by person
!python data_split_person.py

## Load TensorBoard

Now, we set up TensorBoard so that we can graph our accuracy and loss as training proceeds.

In [ ]:
# Load TensorBoard
%load_ext tensorboard
%tensorboard --logdir logs/scalars

## Begin training

The following cell will begin the training process. Training will take around 5 minutes on a GPU runtime. You'll see the metrics in TensorBoard after a few epochs.

In [ ]:
!python train.py --model CNN --person true

## Create a C source file

The `train.py` script writes a quantized model, `model_quantized.tflite`, to the training scripts' directory.

In the following cell, we convert this model into a C++ source file we can use with TensorFlow Lite for Microcontrollers.

In [ ]:
# Install xxd if it is not available
!apt-get -qq install xxd
# Save the file as a C source file
!xxd -i model_quantized.tflite > /content/model_quantized.cc
# Print the source file
!cat /content/model_quantized.cc